In this notebook I attempt to produce reasonable forecasts for the cleaned datasets.

Firstly, I apply a couple of lazy classifer-like packages (Pycaret, skforecast, lazyclassifier) on the dataset. This will enable me to build a lot of basic models without much code and help to quickly understand which models work better without any parameter tuning.

Then after getting the models arranged with respect to their accuracies, I will then choose the top models and apply hyperparameter tuning (optuna/hyperopt).

I will also explore the possibilities of building an assemblage model by applying appropriate bagging techniques with a further aim to reduce model over-fitting.

## PyCaret is an open-source, low-code machine learning library in Python that automates machine learning workflows.

In [23]:
# Importing some of the required packages
from pycaret.time_series import *
import pandas as pd
import numpy as np

In [24]:
# Loading the cleaned dataset. Cleaning was done in a previous notebook
dx = pd.read_csv(r'/home/nkem/Documents/PhD_Research/allN11Oct2022.csv')
dx.head()

,incidentdate,year,month,day,company,contaminant,spillareahabitat,cause,estimatedqty
0,2005-01-05,2005,1,5,MPN,cr,of,ome,0.0568
1,2005-01-08,2005,1,8,MPN,cr,of,eqf,0.0002
2,2005-01-31,2005,1,31,NAOC,cr,la,cor,100.0000
3,2005-02-08,2005,2,8,MPN,cr,of,eqf,0.0300
4,2005-03-08,2005,3,8,MPN,cr,of,eqf,3.0000


In [25]:
# Changing the incidentdate column to pandas datetime
dx['incidentdate'] = pd.to_datetime(dx['incidentdate'])
td = dx.copy()

# Downsampling from daily timeframe to the monthly timeframe
dk = td.groupby([pd.Grouper(key='incidentdate', freq='M')])['estimatedqty'].agg(['sum','size'])
dk = dk.reset_index()

# Renaming the columns
dk.rename(columns={"sum":"estimatedqty", "size":"spillno"}, inplace=True)

# Filtering out the estimatedqty column and dropping the last incomplete entry
df = dk[["incidentdate","spillno"]]
df = df.set_index("incidentdate")
df = df.drop(index="2022-10-31")

# Feauture enginneering - creating month and year column from the incidentdate column
df= df.reset_index()
df["month"] = df["incidentdate"].dt.month
df["year"] = df["incidentdate"].dt.year
dt = df[["incidentdate","spillno","month", "year"]]
dt = dt.set_index("incidentdate")
dt.tail()

,spillno,month,year
incidentdate,,,
2022-05-31,22,5,2022
2022-06-30,48,6,2022
2022-07-31,20,7,2022
2022-08-31,60,8,2022
2022-09-30,38,9,2022


In [29]:
data = dt.copy()
# data split into train & test sets
end_train = '2022-03-31'
start_test = '2022-04-30'
end_test = '2022-06-30'

data_train = dt.loc[: end_train, :]
data_test  = dt.loc[start_test: end_test]

print(f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}")
print(f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}")

Train dates      : 2005-01-31 00:00:00 --- 2022-03-31 00:00:00
Test dates       : 2022-04-30 00:00:00 --- 2022-06-30 00:00:00


In [31]:
# Select exogenous variables, including those generated by one hot encoding.
exog_variables = [column for column in dt.columns
                      if column.startswith(('year', 'month'))]
#exog_variables.extend(['estimatedqty'])
#print(exog_variables)

In [35]:
# For datasets with exogenous variables, a Pycaret experiment demands we explicitly specify the target variable. The Pycaret package helps to divide datasets into training and test datasets so there will not be need for the division above.

target = "spillno"
exog_vars = ['year', 'month']
include = [target] + exog_vars
data = data[include]
data.tail()

,spillno,year,month
incidentdate,,,
2022-05-31,22,2022,5
2022-06-30,48,2022,6
2022-07-31,20,2022,7
2022-08-31,60,2022,8
2022-09-30,38,2022,9


In [37]:
# Forecasting horizon is 3 months
FH= 3
metric = "rmse"

In [38]:
# Global figure settings for notebook
fig_kwargs = {"renderer": "notebook", "width": 1000, "height": 600}

In [43]:
# Pycaret experiment design
 
exp_auto = TSForecastingExperiment()

# enforce_exogenous=False --> Use multivariate forecasting when model supports it, else use univariate forecasting
exp_auto.setup(data=data, target=target, fh=FH, enforce_exogenous=False,fig_kwargs=fig_kwargs, session_id=24,use_gpu=False)

,Description,Value
0,session_id,24
1,Target,spillno
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(213, 3)"
5,Transformed data shape,"(213, 3)"
6,Transformed train set shape,"(210, 3)"
7,Transformed test set shape,"(3, 3)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [44]:
# List of available models
exp_auto.models()

,Name,Reference,Turbo
ID,,,
naive,Naive Forecaster,sktime.forecasting.naive.NaiveForecaster,True
grand_means,Grand Means Forecaster,sktime.forecasting.naive.NaiveForecaster,True
snaive,Seasonal Naive Forecaster,sktime.forecasting.naive.NaiveForecaster,True
polytrend,Polynomial Trend Forecaster,sktime.forecasting.trend.PolynomialTrendForeca...,True
arima,ARIMA,sktime.forecasting.arima.ARIMA,True
auto_arima,Auto ARIMA,sktime.forecasting.arima.AutoARIMA,True
exp_smooth,Exponential Smoothing,sktime.forecasting.exp_smoothing.ExponentialSm...,True
croston,Croston,sktime.forecasting.croston.Croston,True
ets,ETS,sktime.forecasting.ets.AutoETS,True


In [47]:
# Model fiting for 5 mins time budget
# Turbo = False, will ensure that slower models like Prophet will be included
best = exp_auto.compare_models(sort=metric, turbo=False, budget_time=5)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,0.2874,0.2287,6.0782,6.6908,0.2698,0.2395,0.1241,0.0700
snaive,Seasonal Naive Forecaster,0.3149,0.2555,6.6667,7.4769,0.2699,0.2467,-0.1710,0.6467
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,0.3093,0.2641,6.5344,7.7174,0.2588,0.2484,0.0476,1.2700
tbats,TBATS,0.3255,0.2654,6.8733,7.7553,0.2576,0.2602,0.0525,7.2367
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.2625,0.2714,5.5382,7.9223,0.1778,0.1985,0.0163,0.1100
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.3268,0.2783,6.9018,8.1293,0.2607,0.2565,0.0119,0.2733
theta,Theta Forecaster,0.3396,0.2904,7.1748,8.4828,0.2648,0.2673,-0.0645,0.0333
bats,BATS,0.3371,0.2960,7.1295,8.6558,0.2607,0.2702,-0.3275,5.6700
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,0.3658,0.3055,7.7269,8.9266,0.3068,0.2908,-0.2680,0.3367
ets,ETS,0.3623,0.3096,7.6511,9.0410,0.2823,0.2855,-0.1629,0.6900


Processing:   0%|          | 0/137 [00:00<?, ?it/s]

In [51]:
# Increased time budget = 60mins
# Pycaret experiment design
 
exp_auto = TSForecastingExperiment()

# enforce_exogenous=False --> Use multivariate forecasting when model supports it, else use univariate forecasting
exp_auto.setup(data=data, target=target, fh=FH, enforce_exogenous=False,fig_kwargs=fig_kwargs, session_id=25,use_gpu=False)

,Description,Value
0,session_id,25
1,Target,spillno
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(213, 3)"
5,Transformed data shape,"(213, 3)"
6,Transformed train set shape,"(210, 3)"
7,Transformed test set shape,"(3, 3)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [50]:
# Increasing time budget to 60mins
# Turbo = False, will ensure that slower models like Prophet will be included
best = exp_auto.compare_models(sort=metric, turbo=False, budget_time=60)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,0.3088,0.2395,6.5376,7.0081,0.2824,0.2510,-0.0251,0.0667
snaive,Seasonal Naive Forecaster,0.3149,0.2555,6.6667,7.4769,0.2699,0.2467,-0.1710,0.0400
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,0.3093,0.2641,6.5344,7.7174,0.2588,0.2484,0.0476,1.4900
tbats,TBATS,0.3255,0.2654,6.8733,7.7553,0.2576,0.2602,0.0525,7.4000
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.2787,0.2734,5.8783,7.9792,0.1958,0.2176,0.0078,0.1333
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,0.3507,0.2855,7.4038,8.3397,0.2940,0.2785,-0.0988,0.3100
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.3462,0.2901,7.3153,8.4757,0.2877,0.2742,-0.1248,0.2933
theta,Theta Forecaster,0.3396,0.2904,7.1748,8.4828,0.2648,0.2673,-0.0645,0.0367
bats,BATS,0.3371,0.2960,7.1295,8.6558,0.2607,0.2702,-0.3275,5.5400
ets,ETS,0.3623,0.3096,7.6511,9.0410,0.2823,0.2855,-0.1629,0.6800


Processing:   0%|          | 0/137 [00:00<?, ?it/s]

Increasing the time budget in Pycaret didn't improve the accuracies of the models nor the runtime of the experiment